### 演算環境の準備

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import random
random.seed(0)

In [ ]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

## 1. ベースラインモデルの評価

In [ ]:
messages = [
    {"role": "system", "content": "質問に回答してください。必ず「日本語で回答」すること。"},
    {"role": "user", "content": "LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)